# 🌟 Urban Comfort Potential from Open Environmental Data

### Title:
**A reproducible and explainable framework for assessing urban environmental comfort and spatial inequality using open data.**

**Scope**

This notebook implements a fully auditable pipeline to estimate **urban environmental comfort potential** using open datasets only. It builds four structural dimensions:

* **Acoustic exposure** (road noise)
* **Vegetation** (NDVI)
* **Structural crowd pressure** (built and activity proxies)
* **Infrastructure related accessibility** (access and support proxies)

These dimensions are later combined into an interpretable **ComfortScore**.

**Does not:**
* Predict individual wellbeing
* Use personal data, behavioural tracking, or surveys

**Strategy**
* Use OpenStreetMap venues to define the Area of Interest and a consistent measurement support
* Acquire authoritative layers with cache aware scripts
* Save outputs with clear paths and simple provenance logs

## Notebook structure
1. Setup and reproducibility settings
2. Phase A Data acquisition
3. Phase B Feature engineering
4. Phase C ComfortScore model and explanation layer
5. Phase D External data download
6. Phase E External validation and spatial structure
7. Phase-F Final synthesis and future directions


**Setup and reproducibility settings**



In [1]:
# Core
import os
import re
import io
import json
import time
import glob
import zipfile
import hashlib
from pathlib import Path
from datetime import date, datetime, timezone
from concurrent.futures import ThreadPoolExecutor, as_completed

import mgrs
import rasterio
from rasterio.mask import mask
from rasterio.errors import RasterioIOError
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Data
import numpy as np
import pandas as pd
import geopandas as gpd
import requests

import warnings
warnings.filterwarnings("ignore")

print("Setup complete")


Setup complete


# 🎯 Phase-A Data acquisition

**Inputs acquired in Phase A**
* **Venues** from OpenStreetMap to define the Area of Interest
* **DEFRA road noise polygons** via the official OGC API Features endpoint
* **Sentinel 2 NDVI tiles** from CEDA, combined into a summer median composite

**Key reproducibility features**
* All downloads are **cached locally** and skipped on rerun when valid.
* Each dataset writes a **provenance JSON** with timestamps, sources, and key parameters.
* Manual access instructions are printed so a reviewer can verify sources without guessing.


## A0: Paths and study area

In [2]:
# We set a consistent folder structure used across all phases.

# Folder convention:
# `data/raw/` stores downloaded or source like files
# `data/processed/` stores cleaned and standardised outputs used downstream

# Notebook and project paths
NOTEBOOK_DIR = Path(".").resolve()
PROJECT_ROOT = NOTEBOOK_DIR.parent

DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"

RAW_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Study area names as used by OSMnx
PLACE_NAMES = ["London Borough of Camden", "London Borough of Islington"]

# Venue outputs
VENUES_RAW_PATH = RAW_DIR / "venues_raw.geojson"
VENUES_CLEAN_PATH = PROCESSED_DIR / "venues_clean.geojson"

print("Project root:", PROJECT_ROOT)
print("Raw dir:", RAW_DIR)
print("Processed dir:", PROCESSED_DIR)
print("Study area:", PLACE_NAMES)


Project root: /home/sai/test/notebook_test
Raw dir: /home/sai/test/notebook_test/data/raw
Processed dir: /home/sai/test/notebook_test/data/processed
Study area: ['London Borough of Camden', 'London Borough of Islington']


## A1: Venues from OpenStreetMap

We start with a set of candidate venues.

Why venues first:
* They define the **spatial footprint** of where we measure comfort.
* Later features (noise, vegetation, roads, accessibility) are computed around each venue.
* This keeps the pipeline place based and avoids individual level inference.

Outputs:
* `data/raw/venues_raw.geojson`
* `data/processed/venues_clean.geojson`


In [3]:
# Try to import OSMnx
try:
    import osmnx as ox
    OSMNX_OK = True
except Exception as e:
    OSMNX_OK = False
    OSMNX_ERR = str(e)

print("OSMnx available:", OSMNX_OK)
if not OSMNX_OK:
    print("Import error:", OSMNX_ERR)


OSMnx available: True


### A1.1: Venue query and cleaning rules

We use a small set of OSM tags that match common peaceful or inclusive places.

Cleaning rules:
* Reproject to British National Grid for consistent distance calculations
* Keep a small set of stable columns
* Drop empty names to avoid unlabeled fragments
* Drop invalid geometries
* Deduplicate by OSM id
* Add representative point coordinates `x` and `y` for convenience


In [4]:
# Tags define what we treat as candidate venues
VENUE_TAGS = {
    "leisure": ["park", "garden", "nature_reserve"],
    "amenity": ["library", "cafe", "community_centre", "arts_centre", "museum"],
    "landuse": ["grass", "recreation_ground"],
}

# British National Grid for metre based buffers
TARGET_CRS = "EPSG:27700"

def fetch_venues_osm(place_names, tags):
    """Download OSM venues using OSMnx."""
    if not OSMNX_OK:
        raise ImportError("OSMnx is not available. Install osmnx to run this cell.")

    # OSMnx caching helps reproducibility and avoids repeated network calls
    ox.settings.log_console = True
    ox.settings.use_cache = True

    print("Fetching OSM venues for:", place_names)
    t0 = time.time()
    gdf = ox.features_from_place(place_names, tags=tags)
    gdf = gdf.reset_index()
    print("Fetched rows:", len(gdf), "in", round(time.time() - t0, 2), "seconds")
    return gdf

def clean_venues_osm(gdf):
    """Clean raw OSM venues for stable downstream use."""
    gdf = gdf.copy()

    # CRS handling
    if gdf.crs is None:
        gdf = gdf.set_crs("EPSG:4326")
    gdf = gdf.to_crs(TARGET_CRS)

    # Stable id
    if "id" not in gdf.columns:
        if "osmid" in gdf.columns:
            gdf["id"] = gdf["osmid"]
        else:
            gdf["id"] = np.arange(len(gdf))

    keep_cols = [
        "id", "name", "amenity", "leisure", "landuse",
        "wheelchair", "opening_hours", "operator", "geometry"
    ]
    keep_cols = [c for c in keep_cols if c in gdf.columns]
    gdf = gdf[keep_cols].copy()

    # Name cleaning
    gdf["name"] = gdf["name"].fillna("").astype(str).str.strip()
    mask_valid = gdf["name"].apply(lambda x: len(x) > 0 and x.lower() not in ["nan", "none"])
    gdf = gdf[mask_valid].copy()

    # Geometry cleaning
    gdf = gdf[gdf.geometry.notna() & ~gdf.geometry.is_empty].copy()

    # Deduplicate
    gdf["id"] = gdf["id"].astype(str)
    gdf = gdf.drop_duplicates(subset=["id"]).copy()

    # Representative point is safe for polygons and multipolygons
    rep = gdf.geometry.representative_point()
    gdf["x"] = rep.x
    gdf["y"] = rep.y

    print("Cleaned venues:", len(gdf))
    print("CRS:", gdf.crs)
    return gdf

def save_geojson(gdf, path):
    """Save GeoDataFrame to GeoJSON with folders created if needed."""
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    gdf.to_file(path, driver="GeoJSON")
    print("Saved:", str(path))


### A1.2: Run venue acquisition

Cache behaviour:
* If `venues_clean.geojson` exists and is non empty, we reuse it.
* Otherwise we fetch from OSM, save a raw copy, then save a cleaned version.


In [5]:
if VENUES_CLEAN_PATH.exists() and VENUES_CLEAN_PATH.stat().st_size > 0:
    print("Venues already present:", VENUES_CLEAN_PATH)
    venues = gpd.read_file(VENUES_CLEAN_PATH)
    print("Loaded venues:", len(venues))
else:
    venues_raw = fetch_venues_osm(PLACE_NAMES, VENUE_TAGS)
    save_geojson(venues_raw, VENUES_RAW_PATH)

    venues = clean_venues_osm(venues_raw)
    save_geojson(venues, VENUES_CLEAN_PATH)

venues.head()


Fetching OSM venues for: ['London Borough of Camden', 'London Borough of Islington']
Fetched rows: 2555 in 42.01 seconds
Saved: /home/sai/test/notebook_test/data/raw/venues_raw.geojson
Cleaned venues: 1211
CRS: EPSG:27700
Saved: /home/sai/test/notebook_test/data/processed/venues_clean.geojson


,id,name,amenity,leisure,landuse,wheelchair,opening_hours,operator,geometry,x,y
0,13799212,Holborn Library,library,NaN,NaN,yes,"Mo-Th 10:00-19:00, Fr 10:00-17:00, Sa 11:00-17:00",NaN,POINT (530835.223 181922.569),530835.223366,181922.569174
1,25475389,Woburn Cafe,cafe,NaN,NaN,NaN,NaN,NaN,POINT (529873.562 182510.483),529873.561987,182510.483335
2,25497832,Cafe Angel,cafe,NaN,NaN,NaN,NaN,NaN,POINT (530790.094 182331.853),530790.093853,182331.853236
3,248231088,Kalendar,cafe,NaN,NaN,NaN,NaN,NaN,POINT (528356.799 186417.814),528356.798695,186417.813973
4,267971989,The Brew House Café,cafe,NaN,NaN,NaN,Dec-Jan: Mo-Su 09:00-16:00; Feb-Mar: Mo-Su 09:...,English Heritage,POINT (527143.833 187468.195),527143.832630,187468.194588


## A2: DEFRA strategic road noise via OGC API Features

We retrieve DEFRA road noise polygons using the official OGC API Features service.

Why this method is defensible:
* It uses an **official API endpoint** rather than time limited file links.
* The query is bounded to the study area using a venue based Area of Interest.
* The output is stored as a GeoPackage for stable downstream processing.




In [6]:
from datetime import datetime, timezone

# Paths
DEFRA_DIR = RAW_DIR / "defra_noise"
DEFRA_DIR.mkdir(parents=True, exist_ok=True)

DEFRA_OUT_GPKG = DEFRA_DIR / "defra_road_noise_lden_round3_aoi.gpkg"
DEFRA_LAYER = "defra_road_noise_lden_round3_aoi"
DEFRA_PROVENANCE = DEFRA_DIR / "provenance_defra.json"

# AOI buffer expands beyond venues to include local context
AOI_BUFFER_M = 2500

# Set True if you want to force a fresh pull from the API
REFRESH_DEFRA = False

# DEFRA dataset identifiers
DATASET_ID = "fd1c6327-ad77-42ae-a761-7c6a0866523d"
COLLECTION_ID = "Road_Noise_Lden_England_Round_3"

OGC_BASE = f"https://environment.data.gov.uk/geoservices/datasets/{DATASET_ID}/ogc/features/v1"
ITEMS_URL = f"{OGC_BASE}/collections/{COLLECTION_ID}/items"
SOURCE_PAGE_URL = f"https://environment.data.gov.uk/dataset/{DATASET_ID}"

def now_utc_iso():
    return datetime.now(timezone.utc).isoformat()

def load_json_safe(path: Path):
    try:
        if path.exists():
            return json.loads(path.read_text(encoding="utf-8"))
    except Exception:
        pass
    return []

def save_json(path: Path, obj):
    path.write_text(json.dumps(obj, indent=2), encoding="utf-8")

def valid_gpkg(path: Path):
    return path.exists() and path.stat().st_size > 0

def print_manual_defra_instructions():
    print("")
    print("Manual verification instructions")
    print("Source page:", SOURCE_PAGE_URL)
    print("Dataset:", COLLECTION_ID)
    print("Look for a GeoPackage download option if you want to cross check the API output")

def build_aoi_from_venues(venues_path: Path, buffer_m: float):
    v = gpd.read_file(venues_path)
    if v.crs is None:
        raise ValueError("venues CRS missing")
    v = v.to_crs("EPSG:27700")
    try:
        geom = v.geometry.union_all()
    except Exception:
        geom = v.unary_union
    aoi = geom.convex_hull.buffer(float(buffer_m))
    return aoi, aoi.bounds

def fetch_all_features_bbox(bbox_27700, limit=10000, timeout_s=90):
    # Bounding box query in EPSG:27700
    params = {
        "bbox": ",".join([str(x) for x in bbox_27700]),
        "bbox-crs": "http://www.opengis.net/def/crs/EPSG/0/27700",
        "crs": "http://www.opengis.net/def/crs/EPSG/0/27700",
        "limit": int(limit),
        "f": "application/geo+json",
    }

    try:
        from tqdm.auto import tqdm
        use_tqdm = True
    except Exception:
        use_tqdm = False

    features = []
    url = ITEMS_URL
    page = 0

    pbar = None
    if use_tqdm:
        pbar = tqdm(desc="DEFRA pages", unit="page")

    while True:
        page += 1
        r = requests.get(url, params=params, timeout=timeout_s)
        r.raise_for_status()
        js = r.json()

        chunk = js.get("features", [])
        features.extend(chunk)

        next_url = None
        for link in js.get("links", []):
            if link.get("rel") == "next":
                next_url = link.get("href")
                break

        if use_tqdm:
            pbar.update(1)
            pbar.set_postfix({
                "page": page,
                "page_features": len(chunk),
                "total_features": len(features),
            })
        else:
            print("DEFRA page", page, "features", len(chunk), "total", len(features))

        if not next_url:
            break

        # After the first page the next URL usually includes params
        url = next_url
        params = None

        # Safety stop to avoid infinite loops
        if page > 500:
            raise RuntimeError("Too many pages returned")

    if use_tqdm:
        pbar.close()

    return features

def ensure_defra_noise_aoi(venues_path: Path):
    provenance = load_json_safe(DEFRA_PROVENANCE)

    # Cache hit
    if valid_gpkg(DEFRA_OUT_GPKG) and not REFRESH_DEFRA:
        print("DEFRA noise already present, skipping download")
        print("Output file:", DEFRA_OUT_GPKG)
        print_manual_defra_instructions()

        provenance.append({
            "dataset": "DEFRA Road Noise Lden Round 3",
            "method": "ogc_api_features_bbox",
            "status": "already_present",
            "checked_at_utc": now_utc_iso(),
            "output": str(DEFRA_OUT_GPKG)
        })
        save_json(DEFRA_PROVENANCE, provenance)
        return DEFRA_OUT_GPKG

    # Build AOI from venues and query using its bounding box
    _aoi_geom, bbox = build_aoi_from_venues(venues_path, AOI_BUFFER_M)

    t0 = time.time()
    features = fetch_all_features_bbox(bbox)
    if not features:
        raise RuntimeError("No DEFRA features returned")

    gdf = gpd.GeoDataFrame.from_features(features, crs="EPSG:27700")
    gdf = gdf[gdf.geometry.notna() & ~gdf.geometry.is_empty].copy()

    # Overwrite if present
    if DEFRA_OUT_GPKG.exists():
        DEFRA_OUT_GPKG.unlink()

    gdf.to_file(DEFRA_OUT_GPKG, layer=DEFRA_LAYER, driver="GPKG")

    provenance.append({
        "dataset": "DEFRA Road Noise Lden Round 3",
        "method": "ogc_api_features_bbox",
        "status": "downloaded",
        "downloaded_at_utc": now_utc_iso(),
        "source_items_url": ITEMS_URL,
        "source_page": SOURCE_PAGE_URL,
        "aoi_buffer_m": AOI_BUFFER_M,
        "bbox_27700": [float(x) for x in bbox],
        "n_features": int(len(gdf)),
        "runtime_s": float(time.time() - t0),
        "output": str(DEFRA_OUT_GPKG)
    })
    save_json(DEFRA_PROVENANCE, provenance)

    print("DEFRA AOI noise saved")
    print("Total features:", len(gdf))
    print("Output file:", DEFRA_OUT_GPKG)
    print_manual_defra_instructions()
    return DEFRA_OUT_GPKG

# Run acquisition
defra_noise_gpkg = ensure_defra_noise_aoi(VENUES_CLEAN_PATH)
defra_noise_gpkg


DEFRA pages: 0page [00:00, ?page/s]

DEFRA AOI noise saved
Total features: 53137
Output file: /home/sai/test/notebook_test/data/raw/defra_noise/defra_road_noise_lden_round3_aoi.gpkg

Manual verification instructions
Source page: https://environment.data.gov.uk/dataset/fd1c6327-ad77-42ae-a761-7c6a0866523d
Dataset: Road_Noise_Lden_England_Round_3
Look for a GeoPackage download option if you want to cross check the API output


PosixPath('/home/sai/test/notebook_test/data/raw/defra_noise/defra_road_noise_lden_round3_aoi.gpkg')

## A3: Sentinel 2 NDVI from CEDA and summer median composite

We build a robust summer NDVI composite to represent vegetation.

Approach:
* Infer the Sentinel tile code from the venue centroid
* For each target month, select one valid tile image
* Run integrity checks to avoid partial or corrupted TIFF downloads
* Clip each NDVI image to the venue based Area of Interest
* Compute a **median** composite across selected summer images

Note: 
NDVI and strategic road noise layers represent slowly varying structural environmental conditions.
Venue locations are used as spatial anchors to sample these conditions rather than as time specific observations.
The framework therefore estimates structural comfort potential rather than historical realised comfort at a specific year.


In [7]:
# Settings
YEAR = 2017
MONTHS_TARGET = [6, 7, 8]
WORKERS = 6

AOI_BUFFER_M = 2500
CEDA_BASE = "https://dap.ceda.ac.uk/neodc/sentinel_ard/products/indices/sentinel_2/ndvi/v2"
SOURCE_URL = "https://data.ceda.ac.uk/neodc/sentinel_ard/products/indices/sentinel_2/ndvi/v2/"

NDVI_DIR = RAW_DIR / "ndvi"
NDVI_FILES_DIR = NDVI_DIR / "files"
NDVI_DIR.mkdir(parents=True, exist_ok=True)
NDVI_FILES_DIR.mkdir(parents=True, exist_ok=True)

NDVI_PROVENANCE = NDVI_DIR / "provenance_ndvi.json"

# Set True if you want to rebuild the composite
REFRESH_NDVI = False

LIST_TIMEOUT = (10, 25)
DL_TIMEOUT = (20, 900)
CHUNK = 1024 * 1024

def now_utc_iso():
    return datetime.now(timezone.utc).isoformat()

def sha256_file(path: Path):
    h = hashlib.sha256()
    with Path(path).open("rb") as f:
        for chunk in iter(lambda: f.read(1024 * 1024), b""):
            h.update(chunk)
    return h.hexdigest()

def load_json_safely(path: Path):
    try:
        if path.exists():
            return json.loads(path.read_text(encoding="utf-8"))
    except Exception:
        return []
    return []

def save_json(path: Path, obj):
    path.write_text(json.dumps(obj, indent=2), encoding="utf-8")

def valid_file(path: Path):
    return path.exists() and path.stat().st_size > 0

def print_manual_ndvi_instructions(tile_code: str):
    print("")
    print("Manual verification instructions")
    print("Source directory:", f"{SOURCE_URL}/{YEAR:04d}/")
    print("Tile:", f"T{tile_code}" " " "(Note:Inner London, including Camden and Islington, falls inside the Sentinel 2 tile T30UXC.)" )
    print("Target months:", MONTHS_TARGET)
    print("If you need manual download, place NDVI GeoTIFFs into:")
    print(" ", str(NDVI_FILES_DIR))

def make_session():
    s = requests.Session()
    retry = Retry(total=5, backoff_factor=0.5, status_forcelist=[429, 500, 502, 503, 504])
    adapter = HTTPAdapter(max_retries=retry)
    s.mount("https://", adapter)
    s.mount("http://", adapter)
    return s

def get_aoi_geometry_from_venues(venues_path: Path, buffer_m: float):
    v = gpd.read_file(venues_path)
    if v.crs is None:
        raise ValueError("venues CRS missing")
    v = v.to_crs("EPSG:27700")
    try:
        geom = v.geometry.union_all()
    except Exception:
        geom = v.unary_union
    return geom.convex_hull.buffer(float(buffer_m))

def infer_mgrs_tile_from_venues(venues_path: Path):
    v = gpd.read_file(venues_path)
    if v.crs is None:
        raise ValueError("venues CRS missing")
    v = v.to_crs("EPSG:4326")
    try:
        geom = v.geometry.union_all()
    except Exception:
        geom = v.unary_union
    c = geom.centroid
    m = mgrs.MGRS()
    code = m.toMGRS(c.y, c.x, MGRSPrecision=0)
    if isinstance(code, bytes):
        code = code.decode("utf-8")
    return str(code)

def robust_get_text(session: requests.Session, url: str, timeout=LIST_TIMEOUT):
    r = session.get(url, timeout=timeout)
    r.raise_for_status()
    return r.text

def list_tif_links(html: str, base_url: str):
    links = re.findall(r'href="([^"]+\.tif)"', html, flags=re.IGNORECASE)
    out = []
    for lnk in links:
        if lnk.startswith("http"):
            out.append(lnk)
        else:
            out.append(base_url.rstrip("/") + "/" + lnk.lstrip("/"))
    return out

def list_days_in_month(session: requests.Session, year: int, month: int):
    month_url = f"{CEDA_BASE}/{year:04d}/{month:02d}/"
    html = robust_get_text(session, month_url)
    return sorted({int(d) for d in re.findall(r'href="(\d{2})\/"', html)})

def find_all_tile_files_for_day(session: requests.Session, tile: str, year: int, month: int, day: int):
    day_url = f"{CEDA_BASE}/{year:04d}/{month:02d}/{day:02d}/"
    try:
        html = robust_get_text(session, day_url)
        tifs = list_tif_links(html, day_url)
        matches = [u for u in tifs if f"_T{tile}_" in u and "NDVI" in u.upper()]
        if matches:
            return date(year, month, day), matches
    except Exception:
        return None
    return None

def scan_month_for_tile_candidates(session: requests.Session, tile: str, year: int, month: int, workers: int = 6):
    days = list_days_in_month(session, year, month)

    try:
        from tqdm.auto import tqdm
        use_tqdm = True
    except Exception:
        use_tqdm = False

    found = []
    with ThreadPoolExecutor(max_workers=int(workers)) as ex:
        futs = [ex.submit(find_all_tile_files_for_day, session, tile, year, month, d) for d in days]
        it = as_completed(futs)
        if use_tqdm:
            it = tqdm(it, total=len(futs), desc=f"Scan {year}-{month:02d}", unit="day")
        for f in it:
            out = f.result()
            if out:
                found.append(out)

    if not found:
        return []

    found.sort(key=lambda x: x[0])
    candidates = []
    for d, urls in found:
        for u in urls:
            candidates.append((d, u))
    return candidates

def raster_read_test(tif_path: Path):
    """Quick integrity test to catch partial or corrupted TIFFs."""
    if not valid_file(tif_path):
        return False
    try:
        with rasterio.open(tif_path) as src:
            h = int(src.height)
            w = int(src.width)
            if h <= 0 or w <= 0:
                return False
            row0 = max(0, (h // 2) - 8)
            col0 = max(0, (w // 2) - 8)
            window = rasterio.windows.Window(col0, row0, 16, 16)
            _ = src.read(1, window=window, boundless=True)
        return True
    except Exception:
        return False

def download_file_with_progress(session: requests.Session, url: str, out_path: Path):
    """Download a single file with retry and a post download integrity check."""
    out_path = Path(out_path)

    # Cache hit
    if valid_file(out_path) and raster_read_test(out_path):
        return "already_present", sha256_file(out_path)

    # Remove broken file before retry
    if out_path.exists():
        try:
            out_path.unlink()
        except Exception:
            pass

    try:
        from tqdm.auto import tqdm
        use_tqdm = True
    except Exception:
        use_tqdm = False

    with session.get(url, stream=True, timeout=DL_TIMEOUT) as r:
        r.raise_for_status()
        total = r.headers.get("Content-Length")
        total = int(total) if total and total.isdigit() else None

        pbar = None
        if use_tqdm:
            pbar = tqdm(total=total, unit="B", unit_scale=True, desc=out_path.name, leave=False)

        with out_path.open("wb") as f:
            for chunk in r.iter_content(chunk_size=CHUNK):
                if not chunk:
                    continue
                f.write(chunk)
                if pbar is not None:
                    pbar.update(len(chunk))

        if pbar is not None:
            pbar.close()

    # Validate
    if not raster_read_test(out_path):
        try:
            out_path.unlink()
        except Exception:
            pass
        return "download_failed_or_corrupt", None

    return "downloaded", sha256_file(out_path)

def write_median_composite(tif_paths, aoi_geom, out_tif: Path):
    """Clip inputs to AOI and write a median NDVI composite."""
    arrays = []
    meta = None
    bad = []

    for p in tif_paths:
        p = Path(p)
        if not raster_read_test(p):
            bad.append(p.name)
            continue

        try:
            with rasterio.open(p) as src:
                clipped, transform = mask(src, [aoi_geom], crop=True)
                arr = clipped[0].astype(np.float32)

                nodata = src.nodata
                if nodata is not None:
                    arr = np.where(arr == nodata, np.nan, arr)

                arrays.append(arr)

                if meta is None:
                    meta = src.meta.copy()
                    meta.update(
                        height=clipped.shape[1],
                        width=clipped.shape[2],
                        transform=transform,
                        nodata=-9999,
                        dtype="float32",
                        count=1,
                    )
        except (RasterioIOError, Exception):
            bad.append(p.name)
            continue

    if len(arrays) == 0:
        raise RuntimeError("All NDVI inputs failed integrity checks, cannot build composite")

    stack = np.stack(arrays)
    med = np.nanmedian(stack, axis=0)
    med = np.where(np.isfinite(med), med, -9999).astype(np.float32)

    out_tif = Path(out_tif)
    with rasterio.open(out_tif, "w", **meta) as dst:
        dst.write(med, 1)

    return bad

def ensure_ndvi_composite():
    tile = infer_mgrs_tile_from_venues(VENUES_CLEAN_PATH)
    aoi = get_aoi_geometry_from_venues(VENUES_CLEAN_PATH, AOI_BUFFER_M)
    out_tif = NDVI_DIR / f"ndvi_median_summer_{YEAR}_T{tile}_aoi.tif"

    prov = load_json_safely(NDVI_PROVENANCE)

    # Always print manual instructions for auditability
    print_manual_ndvi_instructions(tile)

    # Cache hit
    if valid_file(out_tif) and not REFRESH_NDVI:
        print("")
        print("NDVI composite already present, skipping build")
        print("Output file:", out_tif)
        prov.append({
            "created_at_utc": now_utc_iso(),
            "status": "already_present",
            "year": YEAR,
            "months": MONTHS_TARGET,
            "tile": tile,
            "aoi_buffer_m": AOI_BUFFER_M,
            "output_composite": str(out_tif)
        })
        save_json(NDVI_PROVENANCE, prov)
        return out_tif

    session = make_session()
    downloads = []
    used = []

    for mth in MONTHS_TARGET:
        print("")
        print("Scanning", YEAR, f"{mth:02d}")
        candidates = scan_month_for_tile_candidates(session, tile, YEAR, mth, workers=WORKERS)

        if not candidates:
            print("No tile match found in this month")
            continue

        chosen = None
        for d, url in candidates:
            fname = url.split("/")[-1].split("?")[0]
            out_path = NDVI_FILES_DIR / fname

            status, fhash = download_file_with_progress(session, url, out_path)
            used.append({
                "month": int(mth),
                "selected_date": str(d),
                "url": url,
                "file": fname,
                "status": status,
                "sha256": fhash
            })

            print("Candidate:", d, fname, status)

            if status in ["already_present", "downloaded"]:
                chosen = out_path
                break

        if chosen is None:
            print("All candidates failed for this month")
            continue

        downloads.append(chosen)
        print("Selected for composite:", chosen.name)

    if len(downloads) == 0:
        raise RuntimeError("No valid NDVI files available, cannot create composite")

    print("")
    print("Building median composite from", len(downloads), "images")
    t0 = time.time()
    bad_inputs = write_median_composite(downloads, aoi, out_tif)
    runtime_s = float(time.time() - t0)

    prov.append({
        "created_at_utc": now_utc_iso(),
        "status": "built",
        "year": YEAR,
        "months": MONTHS_TARGET,
        "tile": tile,
        "aoi_buffer_m": AOI_BUFFER_M,
        "downloaded_files": used,
        "output_composite": str(out_tif),
        "output_sha256": sha256_file(out_tif),
        "build_runtime_s": runtime_s,
        "bad_inputs_skipped": bad_inputs
    })
    save_json(NDVI_PROVENANCE, prov)

    print("Saved NDVI composite:", out_tif)
    if bad_inputs:
        print("Skipped corrupted inputs:", bad_inputs)

    return out_tif

ndvi_composite_path = ensure_ndvi_composite()
ndvi_composite_path


print("Note: NDVI is extracted for 2017 to maintain temporal consistency with the DEFRA noise dataset.")


Manual verification instructions
Source directory: https://data.ceda.ac.uk/neodc/sentinel_ard/products/indices/sentinel_2/ndvi/v2//2017/
Tile: T30UXC (Note:Inner London, including Camden and Islington, falls inside the Sentinel 2 tile T30UXC.)
Target months: [6, 7, 8]
If you need manual download, place NDVI GeoTIFFs into:
  /home/sai/test/notebook_test/data/raw/ndvi/files

Scanning 2017 06


Scan 2017-06:   0%|          | 0/18 [00:00<?, ?day/s]

S2A_20170601_lat52lon075_T30UXC_ORB137_utm30n_osgb_NDVI.tif:   0%|          | 0.00/505M [00:00<?, ?B/s]

Candidate: 2017-06-01 S2A_20170601_lat52lon075_T30UXC_ORB137_utm30n_osgb_NDVI.tif downloaded
Selected for composite: S2A_20170601_lat52lon075_T30UXC_ORB137_utm30n_osgb_NDVI.tif

Scanning 2017 07


Scan 2017-07:   0%|          | 0/25 [00:00<?, ?day/s]

S2A_20170731_lat52lon075_T30UXC_ORB137_utm30n_osgb_NDVI.tif:   0%|          | 0.00/516M [00:00<?, ?B/s]

Candidate: 2017-07-31 S2A_20170731_lat52lon075_T30UXC_ORB137_utm30n_osgb_NDVI.tif downloaded
Selected for composite: S2A_20170731_lat52lon075_T30UXC_ORB137_utm30n_osgb_NDVI.tif

Scanning 2017 08


Scan 2017-08:   0%|          | 0/27 [00:00<?, ?day/s]

S2B_20170805_lat52lon075_T30UXC_ORB137_utm30n_osgb_NDVI.tif:   0%|          | 0.00/490M [00:00<?, ?B/s]

Candidate: 2017-08-05 S2B_20170805_lat52lon075_T30UXC_ORB137_utm30n_osgb_NDVI.tif downloaded
Selected for composite: S2B_20170805_lat52lon075_T30UXC_ORB137_utm30n_osgb_NDVI.tif

Building median composite from 3 images
Saved NDVI composite: /home/sai/test/notebook_test/data/raw/ndvi/ndvi_median_summer_2017_T30UXC_aoi.tif
Note: NDVI is extracted for 2017 to maintain temporal consistency with the DEFRA noise dataset.


## A4 Verification summary


In [8]:
# ============================================================
# 4 Verification summary
# ============================================================

def list_gpkgs(folder: Path):
    folder = Path(folder)
    return sorted([p for p in folder.glob("*.gpkg") if p.exists() and p.stat().st_size > 0])

print("Checking outputs")
print("Venues clean exists:", VENUES_CLEAN_PATH.exists(), str(VENUES_CLEAN_PATH))
print("DEFRA gpkg count:", len(list_gpkgs(DEFRA_DIR)), str(DEFRA_DIR))
print("NDVI composite exists:", Path(ndvi_composite_path).exists(), str(ndvi_composite_path))

print("")
print("NDVI composites in folder:")
for p in sorted((RAW_DIR / "ndvi").glob("ndvi_median_summer_*.tif")):
    print(" -", p.name)


Checking outputs
Venues clean exists: True /home/sai/test/notebook_test/data/processed/venues_clean.geojson
DEFRA gpkg count: 1 /home/sai/test/notebook_test/data/raw/defra_noise
NDVI composite exists: True /home/sai/test/notebook_test/data/raw/ndvi/ndvi_median_summer_2017_T30UXC_aoi.tif

NDVI composites in folder:
 - ndvi_median_summer_2017_T30UXC_aoi.tif
